In [1]:
from datasets import load_dataset

dataset = load_dataset("openbookqa", "additional", split=["train", "test"])

In [14]:
NUM_PERMUTE_QUESTION=10
EACH_HAS=24
NUM_CHOICE=4

In [23]:
import random


total_ques=NUM_PERMUTE_QUESTION*EACH_HAS
each_choice=int(total_ques/NUM_CHOICE)
choice_rank=[[] for _ in range(NUM_CHOICE)]
all_index=list(range(total_ques))
random.shuffle(all_index)
for idx in range(NUM_CHOICE):
    choice_rank[idx]=all_index[idx:(idx+1)*each_choice]

In [3]:
dataset[0][0]

{'id': '7-980',
 'question_stem': 'The sun is responsible for',
 'choices': {'text': ['puppies learning new tricks',
   'children growing up and getting old',
   'flowers wilting in a vase',
   'plants sprouting, blooming and wilting'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'D',
 'fact1': 'the sun is the source of energy for physical cycles on Earth',
 'humanScore': 1.0,
 'clarity': 2.0,
 'turkIdAnonymized': 'b356d338b7'}

In [29]:
import numpy as np


def get_prompt(dataset, index, location, has_choice=False):
    json_line = dataset[index]
    question = json_line["question_stem"]
    choices = json_line["choices"]
    choice_texts = choices["text"]
    answer_key = json_line["answerKey"]
    answer_key_idx = ord(answer_key[0]) - ord("A")
    answer_text = choices["text"][answer_key_idx]
    perm=np.random.permutation(len(choice_texts)-1)
    choice_texts.remove(answer_text)
    choice_texts = [choice_texts[perm[i]].replace("\n", " ") for i in range(len(choice_texts))]
    choice_texts.insert(location, answer_text)
    candidates = " ".join(
        [
            f"({label if has_choice else ' '}) {text}"
            for text, label in zip(choice_texts, choices["label"])
        ]
    ).replace("\n", " ")
    
    fact = f"{json_line['fact1']}{'' if json_line['fact1'][-1]=='.' else '.'}"
    prompt = f"{fact} {question} \\n {candidates}"
    return prompt, answer_text

In [30]:
dataset[1][0]

{'id': '8-343',
 'question_stem': 'A person wants to start saving money so that they can afford a nice vacation at the end of the year. After looking over their budget and expenses, they decide the best way to save money is to',
 'choices': {'text': ['make more phone calls',
   'quit eating lunch out',
   'buy less with monopoly money',
   'have lunch with friends'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'B',
 'fact1': 'using less resources usually causes money to be saved',
 'humanScore': 1.0,
 'clarity': 2.0,
 'turkIdAnonymized': 'b356d338b7'}

In [31]:
from tqdm import tqdm, trange
import pickle
DATASET_NAME='obqa_fact'
if NUM_PERMUTE_QUESTION:
    container=[]
    each_choice_has=EACH_HAS/NUM_CHOICE
    for ques_index in trange(NUM_PERMUTE_QUESTION):
        for idx in range(EACH_HAS):
            location=int(idx/each_choice_has)
            container.append(
                    get_prompt(dataset[1],ques_index,location,  has_choice=False)
                )
    pickle.dump((NUM_PERMUTE_QUESTION, EACH_HAS, container), open(f"{DATASET_NAME}_test_permute.pkl", "wb"))
else:
    for index, dataname in enumerate(['train','test']):
        container=[]
        for ques_index, ques in enumerate(tqdm(dataset[index])):
            container.append(
                    get_prompt(dataset[index], ques_index, has_choice=False)
                )
        pickle.dump(container, open(f"{DATASET_NAME}_{dataname}.pkl", "wb"))

100%|██████████| 10/10 [00:00<00:00, 229.01it/s]
